In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models
import cv2
import numpy as np
import math
import os

# --- Step 1: Data Preparation ---
def load_data(data_dir):
    images = []
    landmarks = []
    for file in os.listdir(data_dir):
        if file.endswith(".jpg") or file.endswith(".png"):
            # Load X-ray image
            img_path = os.path.join(data_dir, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))  # Resize for consistency
            images.append(img / 255.0)  # Normalize

            # Load corresponding landmarks
            landmark_file = file.replace(".jpg", ".txt").replace(".png", ".txt")
            landmark_path = os.path.join(data_dir, landmark_file)
            if os.path.exists(landmark_path):
                with open(landmark_path, 'r') as f:
                    landmark = list(map(float, f.read().split()))
                    landmarks.append(landmark)
            else:
                print(f"Warning: Missing landmarks file for {file}. Skipping this image.")
    
    if len(landmarks) == 0:
        raise FileNotFoundError("No valid landmark files found in the dataset directory.")
    
    return np.array(images).reshape(-1, 128, 128, 1), np.array(landmarks)

# --- Step 2: Build the Model ---
def build_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(10)  # Assuming 5 landmarks (x, y) = 10 outputs
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model

# --- Step 3: Train the Model ---
data_dir = r'D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Tracesing_datasets'  # Replace with your dataset path
images, landmarks = load_data(data_dir)

model = build_model()
model.fit(images, landmarks, epochs=50, batch_size=16, validation_split=0.2)

# Save the trained model
model.save("cephalometric_model.h5")

# --- Step 4: Load Model and Predict Landmarks ---
def predict_landmarks(image_path, model_path="cephalometric_model.h5"):
    model = tf.keras.models.load_model(model_path)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    img_resized = cv2.resize(img, (128, 128)) / 255.0
    prediction = model.predict(img_resized.reshape(1, 128, 128, 1))
    if prediction.shape[1] != 10:
        raise ValueError("Prediction does not contain expected 10 landmark values.")
    return prediction.reshape(-1, 2)  # Reshape into [(x1, y1), (x2, y2), ...]

# --- Step 5: Calculate Angles ---
def calculate_angles(landmarks):
    if len(landmarks) < 4:
        raise ValueError("Insufficient landmarks for angle calculation.")
    S, N, A, B = landmarks[0], landmarks[1], landmarks[2], landmarks[3]
    
    def angle_between(p1, p2, p3):
        v1 = np.array([p1[0] - p2[0], p1[1] - p2[1]])
        v2 = np.array([p3[0] - p2[0], p3[1] - p2[1]])
        cosine_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
        angle = np.arccos(cosine_angle)
        return np.degrees(angle)

    SNA = angle_between(S, N, A)
    SNB = angle_between(S, N, B)
    ANB = SNA - SNB

    return {"SNA": SNA, "SNB": SNB, "ANB": ANB}

# --- Step 6: Test the Model ---
image_path = r"D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Percision Smile\Data\p2 gaurav2.jpeg"  # Replace with your test image
landmarks = predict_landmarks(image_path)
angles = calculate_angles(landmarks)

print("Measured Angles:")
print(f"SNA: {angles['SNA']:.2f}°")
print(f"SNB: {angles['SNB']:.2f}°")
print(f"ANB: {angles['ANB']:.2f}°")


FileNotFoundError: No valid landmark files found in the dataset directory.

In [2]:
import os
data_dir = r'D:\COLLEGE\Projects\AI-Driven Cephalometric Analysis\Tracesing_datasets'
files = os.listdir(data_dir)
print(files)  # List all files in the directory to verify landmark file existence


['P44PratikPusam.jpg']


In [6]:
def load_data(data_dir):
    images = []
    landmarks = []
    missing_landmarks = []
    for file in os.listdir(data_dir):
        if file.endswith(".jpg") or file.endswith(".png"):
            # Load X-ray image
            img_path = os.path.join(data_dir, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (128, 128))  # Resize for consistency
            images.append(img / 255.0)  # Normalize

            # Load corresponding landmarks
            landmark_file = file.replace(".jpg", ".txt").replace(".png", ".txt")
            landmark_path = os.path.join(data_dir, landmark_file)
            if os.path.exists(landmark_path):
                with open(landmark_path, 'r') as f:
                    landmark = list(map(float, f.read().split()))
                    landmarks.append(landmark)
            else:
                missing_landmarks.append(file)  # Add to missing list
                
    if len(landmarks) == 0:
        raise FileNotFoundError(f"No valid landmark files found in the dataset directory. Missing landmarks: {missing_landmarks}")
    
    return np.array(images).reshape(-1, 128, 128, 1), np.array(landmarks)

# Test the function
try:
    images, landmarks = load_data(data_dir)
except FileNotFoundError as e:
    print(e)



No valid landmark files found in the dataset directory. Missing landmarks: ['P44PratikPusam.jpg']
